# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [1]:
from utility import * 

Here we use $H_4$ as an example for finding QWC fragments. Notice below that each fragment has the same terms on all qubits.  

To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [3]:
h4 = get_qubit_hamiltonian(mol='h4', geometry=1, basis='sto3g', qubit_transf='jw')

qwc_list = get_qwc_group(h4)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
-0.03364279213332279 [Y0 Z1 Z2 Z3 Y4 Y5 Z6 Y7] +
0.1287476489275114 [Z1 Z2] +
0.09566918130687388 [Z1 Z3] +
-0.008917138825534923 [Z2] +
0.1484641089788795 [Z2 Z3] +
0.09494715992203566 [Z2 Z6] +
-0.008917138825534975 [Z3] +
0.1330052385758922 [Z3 Z6]

Fragments 2:
0.033551136411865544 [Y0 Z1 Y2 X3 Z4 Z5 Z6 X7]

Number of fragments: 25


By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of $H_4$ by partitioning its terms into mutually commuting fragments. 

In [5]:
comm_groups = get_commuting_group(h4)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Number of mutually commuting fragments: 8
The first commuting group
-0.24347573739328326 [] +
-0.019379615157033403 [X0 X1 Y6 Y7] +
0.019379615157033403 [X0 Y1 Y6 X7] +
0.019379615157033403 [Y0 X1 X6 Y7] +
-0.019379615157033403 [Y0 Y1 X6 X7] +
0.13336368496137166 [Z0 Z1] +
0.11120069420693829 [Z0 Z6] +
0.1305803093639717 [Z0 Z7] +
0.1305803093639717 [Z1 Z6] +
0.11120069420693829 [Z1 Z7] +
-0.0027728737704306758 [X2 X3 Y4 Y5] +
0.0027728737704306758 [X2 Y3 Y4 X5] +
0.0027728737704306758 [Y2 X3 X4 Y5] +
-0.0027728737704306758 [Y2 Y3 X4 X5] +
0.1484641089788795 [Z2 Z3] +
0.10669392951173001 [Z2 Z4] +
0.10946680328216069 [Z2 Z5] +
0.10946680328216069 [Z3 Z4] +
0.10669392951173001 [Z3 Z5] +
0.14846410897887957 [Z4 Z5] +
0.1383904089654409 [Z6 Z7]


To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

In [6]:
uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
print(uqwc * uqwc)

This is unitary, U * U^+ = I 
(0.9999999999999999+0j) []


Applying this unitary gives the qubit-wise commuting form of the first mutually commuting group

In [7]:
qwc = remove_complex(uqwc * comm_groups[1] * uqwc)
print(qwc)

-0.24347573739328257 [] +
0.1305803093639712 [X0] +
0.13336368496137127 [X0 X1] +
0.01937961515703348 [X0 X1 Z7] +
0.11120069420693847 [X0 X6] +
-0.01937961515703348 [X0 Z7] +
0.11120069420693847 [X1] +
0.1305803093639712 [X1 X6] +
-0.01937961515703348 [X1 X6 Z7] +
0.10946680328216025 [X2] +
0.14846410897887868 [X2 X3] +
0.0027728737704306723 [X2 X3 Z5] +
0.10669392951173048 [X2 X4] +
-0.0027728737704306723 [X2 Z5] +
0.10669392951173048 [X3] +
0.10946680328216025 [X3 X4] +
-0.0027728737704306723 [X3 X4 Z5] +
0.14846410897887868 [X4] +
0.0027728737704306723 [X4 Z5] +
0.1383904089654414 [X6] +
0.01937961515703348 [X6 Z7]


In addition, current quantum computer can measure only the $z$ operators. Thus, QWC fragments with $x$ or $y$ operators require extra single-qubit unitaries that rotate them into $z$.  

In [8]:
uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999999 []

The all-z form of qwc fragment:
-0.24347573739328252 [] +
0.13058030936397105 [Z0] +
0.1333636849613712 [Z0 Z1] +
0.019379615157033483 [Z0 Z1 Z7] +
0.11120069420693836 [Z0 Z6] +
-0.019379615157033483 [Z0 Z7] +
0.11120069420693836 [Z1] +
0.13058030936397105 [Z1 Z6] +
-0.019379615157033483 [Z1 Z6 Z7] +
0.10946680328216017 [Z2] +
0.14846410897887863 [Z2 Z3] +
0.002772873770430672 [Z2 Z3 Z5] +
0.10669392951173055 [Z2 Z4] +
-0.002772873770430672 [Z2 Z5] +
0.10669392951173055 [Z3] +
0.10946680328216017 [Z3 Z4] +
-0.002772873770430672 [Z3 Z4 Z5] +
0.14846410897887863 [Z4] +
0.002772873770430672 [Z4 Z5] +
0.13839040896544136 [Z6] +
0.019379615157033483 [Z6 Z7]
